In [1]:
import os
import sys
import random
import pstats

import pandas as pd
import hippiehug

sys.path.append('..')

from claimchain import State, View, LocalParams
from claimchain.utils.profiling import Profiler

## Generating test data

This simulates `nb_readers` (200 by default) readers, each with own set of cryptographic keys. Every reader gets a random **32 byte** label associated with them, and **1 Kb** randomly generated claim about them.

The random capability matrix is generated: for each of the target readers _t_, `nb_caps_per_reader` (5 by default) other readers are randomly chosen. These are the ones that the reader _t_ will be able to access claims about, i.e. be included in _t_'s capability list.



In [2]:
def generate_test_data(nb_readers=200, nb_caps_per_reader=5):
    # size = {32, 1024} - len({"label_", "claim_"})
    labels = [b'label_' + os.urandom(32 - 6) for _ in range(nb_readers)]
    claims = [b'claim_' + os.urandom(1024 - 6) for _ in range(nb_readers)]

    params = [LocalParams.generate() for _ in range(nb_readers)]

    reader_graph = {}
    for reader_index in range(nb_readers):
        cap = random.sample(range(nb_readers), nb_caps_per_reader)
        reader_graph[reader_index] = cap

    return reader_graph, labels, claims, params

In [3]:
reader_graph, labels, claims, params = generate_test_data()
len(reader_graph)

200

## Simulating the owner
The owner builds a state consisting of the claims generated above, with the access capabilities as generated above. The state is committed to the chain.

In [9]:
def commit_claims(reader_graph, labels, claims, params):
    owner_params = LocalParams.generate()
    with owner_params.as_default():
        state = State()
        
        # Add claims
        for label, claim in zip(labels, claims):
            state[label] = claim
        
        # Define the capability lists
        for reader_index, cap_indexes in reader_graph.items():
            reader_dh_pk = params[reader_index].dh.pk
            cap_labels = [labels[cap_index] for cap_index in cap_indexes]
            state.grant_access(reader_dh_pk, cap_labels)
        
        # Commit a bunch of times
        chain = hippiehug.Chain()
        state.commit(target_chain=chain)

        return chain, state

## Simulating the readers

In [5]:
def read_claims(chain, reader_graph, labels, claims, params):
    # Go over all of the readers
    for reader_index, caps_indexes in reader_graph.items():
        reader_params = params[reader_index]
        with reader_params.as_default():
            view = View(chain)
            # Retrieve each of the accessible labels
            for cap_index in caps_indexes:
                label = labels[cap_index]
                read_claim = view[labels[cap_index]]
                
                # Check the value matches original to be sure
                assert read_claim == claims[cap_index]

## Measuring timings

In [6]:
profiler = Profiler()
with profiler.as_default():
    # Commit claims
    chain, state = commit_claims(reader_graph, labels, claims, params)
    # Simulate readers
    read_claims(chain, reader_graph, labels, claims, params)

    stats = profiler.compute_stats()

In [7]:
df = pd.DataFrame(stats)
df = df.T
stat_cols = ['avg', 'max', 'min', 'std']
df[stat_cols] = df[stat_cols].apply(lambda value: value * 1000)  # milliseconds
df.sort_values(by='avg')

,avg,max,min,num,std
encode_capability,0.152267,0.507593,0.133753,1000.0,0.037684
_lookup_capability,0.281503,0.753164,0.258923,1000.0,0.030383
encode_claim,1.659107,3.259659,1.202106,200.0,0.376042
decode_claim,2.522745,5.960226,2.071142,1000.0,0.253534
_lookup_claim,2.540730,6.030321,2.085447,1000.0,0.255710
_build_tree,85.764408,85.764408,85.764408,1.0,NaN
